In [6]:
# Kaggle: MNIST Dataset – Number Classification (Local CSV Version)
# End‑to‑end Keras pipeline that loads MNIST CSVs from a local directory

import os
import gc
import random
import numpy as np
import pandas as pd
from typing import Tuple

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ===========================
# 0) Paths & helpers
# ===========================
DATA_DIR = "/content/data"  # Set this to your local directory containing train.csv and test.csv

train_csv = os.path.join(DATA_DIR, "train_mnist.csv")
test_csv = os.path.join(DATA_DIR, "test_mnist.csv")
assert os.path.exists(train_csv) and os.path.exists(test_csv), "train.csv or test.csv not found in DATA_DIR"

# ===========================
# 1) Load data
# ===========================
train = pd.read_csv(train_csv)
test  = pd.read_csv(test_csv)

# Columns: label=int target, id=string identifier, Unnamed: 0=index dump
label_col = "label"
# Feature columns = all pixels like '1x1'..'28x28' (exclude id/label/Unnamed: 0)
drop_cols = {label_col, "id", "Unnamed: 0"}
feature_cols = [c for c in train.columns if c not in drop_cols]

X = train[feature_cols].values.astype("float32")
y = train[label_col].values.astype("int64")
X_test = test[feature_cols].values.astype("float32")

# Normalize 0..255 -> 0..1
X /= 255.0
X_test /= 255.0

# Reshape to images (28x28)
SIDE = int(np.sqrt(X.shape[1]))
assert SIDE * SIDE == X.shape[1], f"Cannot reshape {X.shape[1]} features into a square image."
X = X.reshape(-1, SIDE, SIDE, 1)
X_test = X_test.reshape(-1, SIDE, SIDE, 1)

# Train/valid split
from sklearn.model_selection import train_test_split
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.1, random_state=SEED, stratify=y)

# ===========================
# 2) Model (compact CNN)
# ===========================
model = keras.Sequential([
    layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(SIDE, SIDE, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),

    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2)
]

# Train
model.fit(X_tr, y_tr, validation_data=(X_va, y_va), epochs=15, batch_size=128, callbacks=callbacks, verbose=2)

# Predict
preds = model.predict(X_test, batch_size=512).argmax(axis=1).astype(int)

# ===========================
# 3) Build correct submission (exact columns: id, label)
# ===========================
# `id` must be copied from test (string), `label` must be int 0..9.
sub = pd.DataFrame({
    "id": test["id"].astype(str),
    "label": preds.astype(int)
})

# Save without index
out_path = os.path.join(".", "submission.csv")
sub.to_csv(out_path, index=False)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
211/211 - 60s - 285ms/step - accuracy: 0.8134 - loss: 0.6076 - val_accuracy: 0.1200 - val_loss: 5.2591 - learning_rate: 1.0000e-03
Epoch 2/15
211/211 - 81s - 385ms/step - accuracy: 0.9493 - loss: 0.1693 - val_accuracy: 0.7140 - val_loss: 1.0468 - learning_rate: 1.0000e-03
Epoch 3/15
211/211 - 83s - 394ms/step - accuracy: 0.9665 - loss: 0.1156 - val_accuracy: 0.9683 - val_loss: 0.1146 - learning_rate: 1.0000e-03
Epoch 4/15
211/211 - 80s - 381ms/step - accuracy: 0.9703 - loss: 0.0957 - val_accuracy: 0.9857 - val_loss: 0.0484 - learning_rate: 1.0000e-03
Epoch 5/15
211/211 - 82s - 388ms/step - accuracy: 0.9768 - loss: 0.0739 - val_accuracy: 0.9850 - val_loss: 0.0536 - learning_rate: 1.0000e-03
Epoch 6/15
211/211 - 82s - 389ms/step - accuracy: 0.9795 - loss: 0.0685 - val_accuracy: 0.9873 - val_loss: 0.0398 - learning_rate: 1.0000e-03
Epoch 7/15
211/211 - 83s - 395ms/step - accuracy: 0.9818 - loss: 0.0589 - val_accuracy: 0.9883 - val_loss: 0.0399 - learning_rate: 1.0000e-03
Epoch 